# Diagrama de Especiación en Python

En este notebook vamos a ver como podemos calcular la fracción molar de todas las especies generadas en la disociación de un ácido poliprótico a lo largo de la escala de pH

## CALCULOS PARA EL DIAGRAMA DE ESPECIACION
........................................

Estas funciones, dado un conjunto de pkas, calculan las fracciones
molares de cada especie para cada punto en la escala de pH. 

Se da como ejemplo particular la especiacion de un ácido diprótico
y luego se extiende a la fórmula general de un ácido poliprótico.

Para un ácido poliprótico definido genericamente como $H_NA$, donde N es el
número de protones capaces de ser intercambiados, si se conocen
sus constantes de disociación puede calcularse la fracción molar
de cada una de las especies con distinto grado de disociación para cada pH.

Llamemos P al polinomio definido como

\begin{equation}
P = [H^+]^N + k_1[H^+]^{(N-1)} + k_1k_2[H^+]^{(N-2)}\ + ... +\ k_1k_2...k_N[H^+]^{(N-N\ =\ 0)}
\end{equation}

y Llamamos $T_k$ a cada término del polinomio, de forma tal que

\begin{equation}
P = T_0 + T_1 + T_2 + ... + T_N
\end{equation}

Vemos que el polinomio tiene N+1 términos, es decir, hay un término en el
polinomio por cada especie en el equilibrio. 

Si se trabaja con los balances de masa y las ecuaciones de equilibrio puede
llegarse a la siguiente expresión:

\begin{equation}
X_{H_kA} = \frac{T_{N-k}}{P} \ ;\ donde\ k=0,1,2,...,N
\end{equation}

In [61]:
import numpy as np
import pandas as pd
import plotly.express as px

### Funciones

A continuación definimos las funciones que nos permitirán calcular la fracción molar de cada especie para un dado pH

In [69]:
def generateColNames(protons):
    colNames = []
    for i in range(0,protons+1):
        if(i == 0):
            colNames.append("A")
        elif(i == 1):
            colNames.append("HA")
        else:
            colNames.append("H"+ str(i) + "A")
    return colNames

def constantFromPka(pka):
    return 10**(-pka)

def poliproticSpeciation(pkas, res=10):
    # Ordeno pkas y obtengo ctes
    pkas = sorted(pkas)
    ks = list(map(constantFromPka, pkas))
    N = len(pkas)
    # Genero los nombres de las especies
    colNames = generateColNames(N)
    
    # Genero una lista de valores de pH entre 0 y 14
    pHvals = np.linspace(0, 14, res)
    data = {"pH": pHvals}
    for name in colNames:
        data[name] = []
    # Genero los coeficientes del polinomio
    # [1, k1, k1*k2,...]
    coefs = [1]
    for k in ks:
        coefs.append(coefs[-1]*k)
        
    # calculo las fracciones molares de cada especie:  
    for pH in pHvals:
        # Obtengo la conc de protones al pH dado
        H = 10**(-pH)
        # Calculo los términos del polinomio para ese pH
        pTerms = []
        for i, coef in enumerate(coefs):
            pTerms.append(coef*(H**(N-i)))
        # Uso la fórmula para calcular la fracción
        # molar de cada especie
        for j, name in enumerate(colNames):
            data[name].append(pTerms[N-j]/sum(pTerms))
    return data, colNames

### Cálculos
Reemplaza los siguientes valores con los del ácido de interés

In [70]:
pkas = [2,4,6]
res  =1000

In [71]:
data, colNames = poliproticSpeciation(pkas, res)
df = pd.DataFrame(data = data)
df

,pH,A,HA,H2A,H3A
0,0.000000,9.900980e-13,9.900980e-07,9.900980e-03,9.900980e-01
1,0.014014,1.090383e-12,1.055760e-06,1.022236e-02,9.897766e-01
2,0.028028,1.200814e-12,1.125765e-06,1.055406e-02,9.894448e-01
3,0.042042,1.322414e-12,1.200398e-06,1.089641e-02,9.891024e-01
4,0.056056,1.456311e-12,1.279965e-06,1.124973e-02,9.887490e-01
...,...,...,...,...,...
995,13.943944,1.000000e+00,1.137774e-08,1.294530e-18,1.472883e-30
996,13.957958,1.000000e+00,1.101646e-08,1.213624e-18,1.336984e-30
997,13.971972,1.000000e+00,1.066665e-08,1.137774e-18,1.213624e-30
998,13.985986,1.000000e+00,1.032795e-08,1.066665e-18,1.101646e-30


## Gráfico

In [75]:
fig = px.line(df, x="pH", y=colNames)

fig.update_layout(
    title="Diagrama de especiación",
    xaxis_title="pH",
    yaxis_title="Fracción molar",
    legend_title="Especies",
)
fig.show()